## NumPy

### Généralités

Nous avons brièvement importé le paquet Numpy dans le [tutoriel 4](./Tutorial_4_Imports_functions.ipynb) afin de permettre de diviser une liste d'entiers par un nombre de manière efficace et pratique, alors que ces opérations ne fonctionnent pas sur des objets `list`. Mais NumPy contient beaucoup d'autres choses, notamment : 

* Des définitions pour quasiment toutes les fonctions usuelles, y compris les fonctions hyperboliques et leurs réciproques (par exemple `arctanh`)
* Des méthodes permettant de créer des vecteurs et des matrices, puis ensuite de les manipuler. Les amateurs de MATLAB seront heureux de les retrouver :-)
* Quelques 

### Les objets `np.ndarray`

Nous l'avons brièvement aperçue précédemment. Quand les éléments des `np.ndarray` sont des nombres, les `np.ndarray` représentent des vecteurs, des matrices ou même des tenseurs d'ordre (de dimension) plus élevée. Ils fonctionnent presque comme les listes, et toute la syntaxe associée va donc leur ressembler fortement, __hormis leur comportement avec les opérateurs usuels (+, *, -, / et **) et leur indexation__. On crée de tels objets assez simplement par exemple à partir d'une liste et de la fonction `np.array()`: 

In [ ]:
import numpy as np
x = np.array([3,4,5])

print(x)
print(type(x))

#### Indexation et _coupes_ /!\\

Les `np.ndarray` s'indexent d'une manière différente que les objets `list` en Python. Pour les tableaux à une dimension, toutefois, rien de bien méchant, on peut facilement récupérer un élément, ou une _coupe_ ([cf Tutoriel 2](./Tutorial_2_ListsTuplesDicts.ipynb#Coupes)) et les résultats sont identiques à ceux obtenus pour les listes :

In [ ]:
my_list = [1,2,6,9,11,-3]
my_array = np.array(my_list)
print(my_list[5])
print(my_array[5])
print(my_list[2:4])
print(my_array[2:4])

Maintenant, _supposons_ que j'aie une liste de 3 listes contenant chacune 3 nombres appelée `mylist_of_lists` (cf ci-dessous). Ma liste de listes peut être vue comme une matrice $3 \times 3$. Je voudrais faire une _coupe_  de cette matrice et ne garder que le bloc $2\times 2$ correspondant au nombres 'en bas à droite' de la matrice initiale. 

On peut assez facilement sélectionner les deux dernières lignes de la liste de liste avec la _coupe_ `[1:3]`, mais il n'est pas possible de directement sélectionner certaines colonnes de la liste. Voyez plutôt : 

In [ ]:
mylist_of_lists = [[1,7,2],[3,4,5],[6,7,8]]

print(mylist_of_lists[1:3])                                # Almost, but not quite
print(mylist_of_lists[1:3][1:3])                           # Does not work

Pour faire une sélection sur les colonnes, il nous faudra examiner un par les deux derniers élements de la liste de listes (c'est à dire les deux dernières lignes de la matrice $3\times 3$) et pour chacune de ces deux lignes (ou _sous-listes_), ne garder que les deux derniers nombres. On ne peut effectuer une telle opération qu'avec une boucle `for` :

In [ ]:
mylist_of_lists = [[1,7,2],[3,4,5],[6,7,8]]
sub_list_of_lists = []
for row in mylist_of_lists[1:]:
    sub_list_of_lists.append(row[1:])
print(sub_list_of_lists)                                           # Too Complicated ! 

_NumPy_ va ici grandement nous aider, car il permet de faire des coupes selon plusieurs dimensions en utilisant la syntaxe 
```
sub_matrix = my_matrix[a:b,c:d,e:f, ...]
```
Ce qui va sélectionner 
* les _lignes_ $a$ à $b-1$ dans la première dimension
* les _colonnes_ $c$ à $d-1$ dans la deuxième dimension
* les _couches_ $e$ à $f-1$ dans la troisième dimension
* etc. etc. pour les dimensions suivantes 

Pour le cas de l'exemple précédent : 

In [ ]:
mylist_of_lists = [[1,7,2],[3,4,5],[6,7,8]]
my_matrix = np.array(mylist_of_lists)
print(my_matrix[1:3,1:3])   # Quick and easy ! 

#### Sommes et multiplications /!\\

__Contrairement__ aux listes à nouveau, la multiplication et l'addition de scalaires à un `np.ndarray` fonctionnent également comme sous MATLAB, et ce quel que soit la 'dimension' du tableau: 
* Pour `+` et `-`, on ajoute (soustrait) donc à chacun des éléments du tableau le scalaire en question
* Pour `*` et `/`, on multiplie (divise) donc chacun des éléments du tableau par le scalaire en question
* Pour `**`, on met chacun des éléments du tableau à la puissance en question (__le `.^` de MATLAB__ !)

In [ ]:
vector = np.array([3,6,1])
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(vector + 3.15)
print(-5.7*vector)
print(-2.1*matrix + 6.32)
print(matrix**2)

Quand les tableaux ont des dimensions compatibles, on peut les multiplier entre eux, et par défaut cette multiplication se fait _élément par élément_ (le `.*` préféré des amateurs de MATLAB) : 


In [ ]:
vector_1 = np.array([3,6,2])
vector_2 = np.array([6,4,1])
matrix = np.array([[1,1,1],[1,1,1],[1,1,1],[2,2,2]])

print(vector_1*vector_2)
print(vector_1*matrix)

* Dans le premier cas, le comportement est assez logique, on multiplie les éléments de chaque vecteur 'un par un'.
* Dans le deuxième cas, NumPy a automatiquement cherché une dimension de `matrix` dont la longueur (3) est compatible avec celle (3) de `vector_1`, et ici, c'est la dimension n°2 (les colonnes). NumPy va ensuite _étendre_ `vector_1` dans l'autre dimension (ou les autres dimensions) pour obtenir la même forme que `matrix`, et enfin multiplier un par un les éléments de `matrix` et de `vector_1` étendu. 

#### Opérations sur les dimensions

##### Listage

On peut demander à NumPy de renvoyer les propriétés de forme des objets `np.ndarray` en utilisant les fonctions : 
* `np.shape()` : va renvoyer la longueur d'un tableau dans chacune des dimensions de celui-ci
* `np.size()` : va renvoyer le nombre d'éléments total du tableau
* `np.ndim()` : va renvoyer le nombre de dimensions du tableau 

In [ ]:
tensor = np.array([[[1,2],[2,3]],[[12,14],[15,19]],[[-2,-1],[-3,-6]]])
print(np.shape(tensor))
print(np.size(tensor))
print(np.ndim(tensor))

__Mini-quiz__: 
* Que renvoie la commande `len(tensor)` ? Comment interprétez-vous le résultat ?
* Essayez de re-créer les fonctions `np.size()` et `np.ndim()` à partir de `np.shape()` en utilisant des fonctions Python et des fonctions Numpy, par ex. `np.sum()`

##### Rajouter des dimensions

On peut assez facilement rajouter des dimensions en exécutant la fonction `np.atleast_nd()` (avec $n = 1,2,3$). En fonction de la dimension de votre tableau initial, les dimensions _en plus_ ne seront pas forcément rajoutées là où vous vous y attendez ! Voyez par exemple, pour un tableau à une dimension (un vecteur) transformé en tableau à trois dimensions :

In [ ]:
x = np.array([1,2,6,8,2])
y = np.atleast_2d(x)
z = np.atleast_3d(x)
print(np.shape(x))
print(np.shape(y))
print(np.shape(z))

Vous pouvez également rajouter _à la main_ des dimensions en plus dans votre tableau en l'indexant avec une coupe qui renverrait normalement la matrice entière `[:,...,:]` et en rajoutant un `np.newaxis` à l'endroit de la dimension que vous souhaitez rajouter. Par exemple, on peut ici rajouter une troisième dimension à une matrice 'en premier', ou même 'au milieu' de la matrice initiale !

In [ ]:
matrix = np.array([[1,2,3],[6,5,4]])
bigger_matrix = matrix[:,np.newaxis,:]
other_bigger_matrix = matrix[np.newaxis,:,:]
print(np.shape(bigger_matrix))
print(np.shape(other_bigger_matrix))

##### Permuter des dimensions

Vous pouvez également permuter certaines dimensions, et les déplacer avec les fonctions :
* `np.moveaxis()` qui déplace un ou plusieurs axes vers une ou plusieurs positions données,
* `np.swapaxes()` qui échange deux listes d'axes
* `np.transpose()` qui, enfin, inverse les axes (les axes 0,1,2,...,n deviennent les axes n,n-1,...,0). La fonction `np.transpose()` fonctionne comme la transposée _normale_ pour des tableaux de dimension 2.

Le résultat fait souvent mal à la tête :-), voyez donc :  

In [7]:
tensor = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]],[[13,14,15],[16,17,18]],[[19,20,21],[22,23,24]]])

print('Initial, shape = ' + str(np.shape(tensor)))
print(tensor)

moved = np.moveaxis(tensor, 1, -1) # Moving the second axis (index 1, columns) to third axis (index -1, layers)
print('-------')
print('Moved, shape = ' + str(np.shape(moved)))
print(moved)

swapped = np.swapaxes(tensor, 0,1) # Exchanging the first (index 0, rows) and the last axes (index -1, layers)
print('-------')
print('Swapped, shape = ' + str(np.shape(swapped)))
print(swapped)

transposed = np.transpose(tensor) # NOTE : tensor.T also works
print('-------')
print('Transposed, shape = ' + str(np.shape(transposed)))
print(transposed)

Initial, shape = (4, 2, 3)
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]

 [[19 20 21]
  [22 23 24]]]
-------
Moved, shape = (4, 3, 2)
[[[ 1  4]
  [ 2  5]
  [ 3  6]]

 [[ 7 10]
  [ 8 11]
  [ 9 12]]

 [[13 16]
  [14 17]
  [15 18]]

 [[19 22]
  [20 23]
  [21 24]]]
-------
Swapped, shape = (2, 4, 3)
[[[ 1  2  3]
  [ 7  8  9]
  [13 14 15]
  [19 20 21]]

 [[ 4  5  6]
  [10 11 12]
  [16 17 18]
  [22 23 24]]]
-------
Transposed, shape = (3, 2, 4)
[[[ 1  7 13 19]
  [ 4 10 16 22]]

 [[ 2  8 14 20]
  [ 5 11 17 23]]

 [[ 3  9 15 21]
  [ 6 12 18 24]]]


_Note_ : Pour les habitués de MATLAB, la fonction `np.transpose()` ne fonctionne pas avec les vecteurs. En effet, sous MATLAB, tous les objets ont au moins 2 dimensions par défaut, mais pas sous NumPy, où les objets à 0 et 1 dimensions existent vraiment. Pour effectuer une réelle transposition d'un vecteur, il faudra utiliser _d'abord_ la fonction `np.atleast_2d()` sur votre vecteur pour que la transposition se fasse correctement.

In [8]:
vector = np.array([1,2,6,2,1])

print('Original, shape = ' + str(np.shape(vector)))
print(vector)

print('Transposed --- Naive, shape : ' + str(np.shape(np.transpose(vector))))
print(np.transpose(vector))

print('Transposed --- OK, shape : ' + str(np.shape(np.transpose(np.atleast_2d(vector)))))
print(np.transpose(np.atleast_2d(vector)))

Original, shape = (5,)
[1 2 6 2 1]
Transposed --- Naive, shape : (5,)
[1 2 6 2 1]
Transposed --- OK, shape : (5, 1)
[[1]
 [2]
 [6]
 [2]
 [1]]


##### Redimensionner des `np.ndarray`

Vous pouvez enfin redimensionner les tableaux en question. Par exemple, vous pouvez transformer une matrice $n \times m$ en une autre matrice de taille $p \times q$, à condition bien entendu que $n\times m = p\times q$. Pour cela, on utilisera la fonction `np.reshape()`. Celle-ci prend en première entrée le tableau à redimensionner, et en deuxième les entiers correspondant à la nouvelle forme attendue pour la matrice. 

_Quelques remarques_

* Les nouvelles dimensions doivent être transmises sous forme de liste ou de tuple, c'est pourquoi on doit mettre ces dimensions entre parenthèses `(`,`)` ou crochets `[`,`]`.
* Il est possible de rajouter une dimension au passage, si $n \times m = p \times q \times r$ ! En supprimer une est également possible.
* Si l'ordre dans lequel les éléments de la matrice ne vous conviennent pas, vous pouvez toujours essayer de la _transposer_ ou de _bouger ses dimensions_ avant de la redimensionner !

In [11]:
my_matrix = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

print(my_matrix)
print(np.reshape(my_matrix,[2,6]))          
print(np.reshape(np.transpose(my_matrix),[2,6]))
print(np.reshape(my_matrix, [2,2,3]))

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]
[[ 1  4  7 10  2  5]
 [ 8 11  3  6  9 12]]
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]


Vous pouvez enfin aplatir un tableau complètement de deux manières différentes. Pour cela, vous pourrez utiliser les _méthodes_ `.flatten()` ou `.ravel()` disponibles pour les tableaux `np.ndarray`. La différence entre ces deux méthodes est que l'objet créé par `.ravel()` va rester lié à la matrice initiale, et va par exemple permettre de la mettre à jour, tandis que celui créé par `.flatten()` sera complètement distinct de la matrice initiale. C'est du chinois ? Allez voir le [Tutoriel 2](./Tutorial_2_ListsTuplesDicts.ipynb#Python-et-la-mutabilité) !

In [ ]:
my_matrix = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

flattened = my_matrix.flatten()
unraveled = my_matrix.ravel()

print('Flattened :')
print(flattened)

print('Unraveled : ')
print(unraveled)

print('-Modifying flattened-')
flattened[3] = -5
print(my_matrix) # my_matrix not impacted by changes of flattened

print('-Modifying Unraveled-')
unraveled[3] = -7
print(my_matrix) # my_matrix not impacted by changes of flattened


#### Remplir des `np.ndarray` efficacement

##### Vecteurs

Remplir les tableaux avec la commande `np.array()` est particulièrement fastidieux. Il serait par exemple intéressant de pouvoir emplir un vecteur avec des valeurs régulièrement espacées entre deux valeurs $a$ et $b$

Obtenir des valeurs espacées régulièrement entre $a$ et $b$ est possible, et ce de plusieurs manières, grâce aux fonctions : 

* `np.arange(a, b, r)`, qui crée un tableau de valeurs correspondant à $a, a+r, a+2r, \ldots ...$ jusqu'à atteindre $b$ (exclus)
* `np.linspace(a, b, N)`, qui crée un tableau 1d contenant $N$ valeurs régulièrement espacées linéairement entre $a$ et $b$ (__les deux inclus__)
* `np.logspace(a, b, N)`, qui crée un tableau 1d contenant $N$ valeurs régulièrement et __logarithmiquement__ espacées entre $\log_{10}(a)$ et $\log_{10}(b)$. 

Ces fonctions sont particulièrement pratiques pour créer des valeurs d'abscisse (l'axe $x$ des graphes) sur lesquels vous allez ensuite appliquer des fonctions mathématiques pour obtenir des valeurs d'ordonnée (l'axe $y$ des graphes).

Voyez plutôt : 

In [18]:
print(np.arange(1.2,23,3.5))
print(np.linspace(0,24,13))
print(np.logspace(-5,0,11))

[ 1.2  4.7  8.2 11.7 15.2 18.7 22.2]
[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24.]
[1.00000000e-05 3.16227766e-05 1.00000000e-04 3.16227766e-04
 1.00000000e-03 3.16227766e-03 1.00000000e-02 3.16227766e-02
 1.00000000e-01 3.16227766e-01 1.00000000e+00]


##### Matrices 

De nombreuses fonctions permettent de créer des matrices de tailles arbitraires. 

* La plus simple d'entre elles, `np.zeros()`, va créer une matrice remplie de zéros de la taille qui nous intéresse. Cette taille sera spécifiée sous la forme d'une liste ou de tuple. 
* De la même manière, `np.ones()` va créer une matrice remplie de 1, en lui spécifiant en entrée une liste de dimensions.
* La fonction `np.eye()` va créer une matrice identité de taille donnée. Cette fois-ci, comme c'est une matrice identité, on ne lui donne en entrée qu'un entier.

Cela donne : 

In [25]:
requested_size = (3,2)
print(np.zeros(requested_size))
print(np.ones([4,2]))
print(np.eye(5))


[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
